<a href="https://colab.research.google.com/github/LeoFernanndes/notebooks/blob/master/hackathon_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
"""

24 metricas em modelos de classificação
https://neptune.ml/blog/evaluation-metrics-binary-classification


"""

'\n\n24 metricas em modelos de classificação\nhttps://neptune.ml/blog/evaluation-metrics-binary-classification\n\n\n'

In [0]:
def classificador_hackathon(df_prev):
    
    '''
    funcao a ser usada para classificar as entradas em  uma base de dados de um sistema de oferta de crédito

    df_prev: DataFrame a com as entradas a terem sua prepensao ao aceite prevista
    
    '''
    

    # subindo o DataFrame que vai ser usado para treinar o modelo
    import pandas as pd 
    
    ulr_full = 'https://raw.githubusercontent.com/LeoFernanndes/datasets/master/bank_full.csv'
    url = 'https://raw.githubusercontent.com/LeoFernanndes/datasets/master/bank.csv' 
    
    # dados de treino e dados de teste
    data_full = pd.read_csv(ulr_full, sep= ';', encoding= 'utf-8')
    data = pd.read_csv(url, sep= ';', encoding= 'utf-8')
    
    # imprimindo os nomes das colunas disponiveis para facilitar consulta e manipulação
    ref = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome']
    
    colunas = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome']
    
    # separando um pedaço do bank_full pra usar como dados de treino
    data_full_treino = data
    data_full_teste = data_full
    
    # usando o label encoder para transformar as variaveis categorias em numericas e assim poder treinar o modelo
    from sklearn.preprocessing import LabelEncoder
    atributos = data_full_treino.copy()[colunas]
    rotulos = data_full_treino.copy()['y']
    
    label_encoder = LabelEncoder()
    for column in atributos.columns:
        if atributos[column].dtype == 'object':
            atributos[column] = label_encoder.fit_transform(atributos[column])        
    
    # chamando o classificador
    from sklearn.ensemble import RandomForestClassifier
    clf = RandomForestClassifier()
    clf.fit(atributos, rotulos)
    
    
    from sklearn.model_selection import cross_val_score
    from sklearn.metrics import confusion_matrix
    
    # avaliação de resultados
    pontuacao = cross_val_score(clf, atributos, rotulos, cv=10, scoring = "accuracy")
    print('Minimo', pontuacao.min(), '\n')
    print('Média ', pontuacao.mean(), '\n')
    print('Maximo', pontuacao.max(), '\n')
    print('Desvio padrão ', pontuacao.std(), '\n')
    
    
    
    df_prev1 = df_prev.copy()[colunas]
    for column in df_prev.columns:
        if df_prev1[column].dtype == 'object':
            df_prev1[column] = label_encoder.fit_transform(df_prev1[column])
    
    
    resultado = clf.predict(df_prev1)
    
    featimp = pd.Series(clf.feature_importances_, index= atributos.columns).sort_values(ascending=False)
    
    matrix = confusion_matrix(resultado, data_full_teste['y'])

    print(featimp, '\n')
    print('Confusion matrix \n', matrix, '\n')
    
    precision = matrix[1][1] / (matrix[0][1] + matrix[1][1])
    print(precision)
    """
    # precisão, recall e f1 para a avaliação da utilidade do modelo 
    recall = 1827 / (1827 + 3462)
    precision = 1827 / (1827 + 788)
    f1 = 2 * recall * precision / (recall + precision)
    print('Precision \n', precision, '\n', 'Recall \n', recall, '\n', 'F1 score \n', f1)
    """

    #resultado1 = pd.DataFrame(resultado.astype(object), columns= ['accepts'])
    #final = df_prev.append(resultado1)
    
    return resultado
    

In [0]:
import pandas as pd

data_full = pd.read_csv('https://raw.githubusercontent.com/LeoFernanndes/datasets/master/bank_full.csv', sep= ';', encoding= 'utf-8')
entrada = data_full.drop(['y'], axis= 1)

In [0]:
previsao = classificador_hackathon(entrada)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Minimo 0.8805309734513275 

Média  0.8940504795952254 

Maximo 0.911504424778761 

Desvio padrão  0.008415010265735066 

duration     0.283623
balance      0.113470
age          0.101043
month        0.094673
day          0.092602
pdays        0.053224
job          0.046659
poutcome     0.045877
marital      0.031475
campaign     0.031272
previous     0.025477
education    0.025223
housing      0.024498
contact      0.018648
loan         0.009056
default      0.003179
dtype: float64 

Confusion matrix 
 [[39001  3407]
 [  921  1882]] 

0.35583286065418795


In [0]:
## Backup da função
## analisando o modelo para aumentar o f1 e implementar uma melhor solução dentro da função de classificação

 # subindo o DataFrame que vai ser usado para treinar o modelo
import pandas as pd 

ulr_full = 'https://raw.githubusercontent.com/LeoFernanndes/datasets/master/bank_full.csv'
data_full = pd.read_csv(ulr_full, sep= ';', encoding= 'utf-8')

# imprimindo os nomes das colunas disponiveis para facilitar consulta e manipulação
ref = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
    'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
    'previous', 'poutcome']

# colunas a serem usadas no modelo
colunas = ['duration', 'age', 'balance', 'day', 'month', 'poutcome', 'job']

# separando um pedaço do bank_full pra usar como dados de treino
data_full_treino = data_full

# usando o label encoder para transformar as variaveis categorias em numericas e assim poder treinar o modelo
from sklearn.preprocessing import LabelEncoder
atributos = data_full_treino.copy()[colunas]
rotulos = data_full_treino.copy()['y']

label_encoder = LabelEncoder()
for column in atributos.columns:
    if atributos[column].dtype == 'object':
        atributos[column] = label_encoder.fit_transform(atributos[column])        

# dividindo o dataset em treino e teste
from sklearn.model_selection import train_test_split
x_treino, x_teste, y_treino, y_teste = train_test_split(atributos, rotulos, train_size= 0.20, random_state= 55)

# chamando o classificador
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators= 10)
clf.fit(x_treino, y_treino)

# validação cruzada
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score

pontuacao = cross_val_score(clf, atributos, rotulos, cv=10, scoring = "accuracy")

print('Desempenho Cross Validation \n',
      'Minimo', pontuacao.min(), '\n',
      'Média ', pontuacao.mean(), '\n',
      'Maximo', pontuacao.max(), '\n',
      'Desvio padrão ', pontuacao.std(), '\n')

# ajusto do modelo aos dados de treino para previsão
resultado = clf.predict(x_teste)

# lista dos atributos mais relevantes 
featimp = pd.Series(clf.feature_importances_, index= atributos.columns).sort_values(ascending=False)
print('Feature importance \n',
      featimp, '\n')

# confusion matrix 
matrix = confusion_matrix(resultado, y_teste)
print('Confusion matrix \n', matrix, '\n')

# metricas de desempenho
accuracy = accuracy_score(resultado, y_teste)
precision = matrix[1][1] / (matrix[1][0] + matrix[1][1])
recall = matrix[1][1] / (matrix[0][1] + matrix[1][1])
f1_score = 2 * precision * recall / (precision + recall)
print('Accuracy \n', accuracy, '\n',
      'Precision \n', precision, '\n',
      'Recall \n', recall, '\n',
      'F1 score \n', f1_score, '\n')

# verificando os valores reais
aceite = data_full.loc[(data_full['y'] == 'yes')]['y'].count()
print('Proporção real aceita \n',
      aceite/len(data_full), '\n',
      'proporção real rejeita \n',
      1 - aceite/len(data_full))

Desempenho Cross Validation 
 Minimo 0.46936518469365185 
 Média  0.7592769258847911 
 Maximo 0.8730650154798761 
 Desvio padrão  0.11843804502583768 

Feature importance 
 duration    0.316447
balance     0.149602
age         0.144631
day         0.127233
month       0.120240
poutcome    0.074691
job         0.067157
dtype: float64 

Confusion matrix 
 [[30963  2749]
 [ 1030  1427]] 

Accuracy 
 0.8955182614946501 
 Precision 
 0.5807895807895808 
 Recall 
 0.3417145593869732 
 F1 score 
 0.4302728780340721 

Proporção real aceita 
 0.11698480458295547 
 proporção real rejeita 
 0.8830151954170445


In [0]:
## analisando o modelo para aumentar o f1 e implementar uma melhor solução dentro da função de classificação

# subindo o DataFrame que vai ser usado para treinar o modelo
import pandas as pd 

ulr_full = 'https://raw.githubusercontent.com/LeoFernanndes/datasets/master/bank_full.csv'
data_full = pd.read_csv(ulr_full, sep= ';', encoding= 'utf-8')

# imprimindo os nomes das colunas disponiveis para facilitar consulta e manipulação
ref = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
    'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
    'previous', 'poutcome']

# colunas a serem usadas no modelo
colunas = ['duration', 'age', 'balance', 'day', 'month', 'poutcome', 'job']

# separando um pedaço do bank_full pra usar como dados de treino
data_full_treino = data_full

# usando o label encoder para transformar as variaveis categorias em numericas e assim poder treinar o modelo
from sklearn.preprocessing import LabelEncoder
atributos = data_full_treino.copy()[colunas]
rotulos = data_full_treino.copy()['y']

label_encoder = LabelEncoder()
for column in atributos.columns:
    if atributos[column].dtype == 'object':
        atributos[column] = label_encoder.fit_transform(atributos[column])        

# dividindo o dataset em treino e teste
from sklearn.model_selection import train_test_split
x_treino, x_teste, y_treino, y_teste = train_test_split(atributos, rotulos, train_size= 0.20, random_state= 55)

# chamando o classificador
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators= 10)
clf.fit(x_treino, y_treino)

# validação cruzada
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score

# esse modelo de avaliação de cross validation é pouco interessante, mas ja indica que o modelo é pior que chutar não pra tudo
# para avaliar recall e precision, provavelmente tenho que mudar as labels de ['y'] para numéricas
pontuacao1 = cross_val_score(clf, atributos, rotulos, cv=10, scoring = "accuracy")

print('Desempenho Cross Validation \n',
      'Minimo', pontuacao1.min(), '\n',
      'Média ', pontuacao1.mean(), '\n',
      'Maximo', pontuacao1.max(), '\n',
      'Desvio padrão ', pontuacao1.std(), '\n')

# ajuste do modelo aos dados de treino para previsão
resultado = clf.predict(x_teste)

# lista dos atributos mais relevantes 
featimp = pd.Series(clf.feature_importances_, index= atributos.columns).sort_values(ascending=False)
print('Feature importance \n',
      featimp, '\n')

# confusion matrix 
matrix = confusion_matrix(resultado, y_teste)
print('Confusion matrix \n', matrix, '\n')

# metricas de desempenho
accuracy = accuracy_score(resultado, y_teste)
precision = matrix[1][1] / (matrix[1][0] + matrix[1][1])
recall = matrix[1][1] / (matrix[0][1] + matrix[1][1])
f1_score = 2 * precision * recall / (precision + recall)
print('Accuracy \n', accuracy, '\n',
      'Precision \n', precision, '\n',
      'Recall \n', recall, '\n',
      'F1 score \n', f1_score, '\n')

# verificando os valores reais
aceite = data_full.loc[(data_full['y'] == 'yes')]['y'].count()
print('Proporção real aceita \n',
      aceite/len(data_full), '\n',
      'proporção real rejeita \n',
      1 - aceite/len(data_full))

Desempenho Cross Validation 
 Minimo 0.46228710462287104 
 Média  0.7641218139316333 
 Maximo 0.8679787704555506 
 Desvio padrão  0.11682473162179623 

Feature importance 
 duration    0.329347
balance     0.150184
age         0.145885
day         0.120411
month       0.114210
poutcome    0.077749
job         0.062214
dtype: float64 

Confusion matrix 
 [[30974  2768]
 [ 1019  1408]] 

Accuracy 
 0.8952970776078962 
 Precision 
 0.5801400906468892 
 Recall 
 0.3371647509578544 
 F1 score 
 0.42647281538694537 

Proporção real aceita 
 0.11698480458295547 
 proporção real rejeita 
 0.8830151954170445
